# Pomer dát 9:1

Súbor: train_9_test_1.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [5]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 
from sklearn.model_selection import train_test_split

Načítanie dát

In [6]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [7]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [8]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [9]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

In [10]:
X = np.concatenate([x_train,x_test])

In [11]:
Y = np.concatenate([y_train,y_test])

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.1)

In [10]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(42742, 128, 128, 1)
(4750, 128, 128, 1)
(42742, 2)
(4750, 2)


CNN

In [11]:
model = Sequential()

In [12]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [13]:
model.add(Conv2D(64, 3, strides=1, activation='relu')) 

In [14]:
model.add(MaxPooling2D(3, 2))

In [15]:
model.add(Conv2D(64, 5, strides=1, padding='same', activation='relu'))

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [18]:
model.add(Conv2D(64, 3, strides=1, padding='same', activation='relu'))

In [19]:
model.add(MaxPooling2D(3, 2))

In [20]:
model.add(Flatten())

In [21]:
model.add(Dense(96, activation='relu'))

In [22]:
model.add(Dropout(0.25)) 

In [23]:
model.add(Dense(96, activation='relu'))

In [24]:
model.add(Dropout(0.25)) 

In [25]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [26]:
adam = Adam()

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [28]:
saved_model = "model_9_1.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

test loss je chyba klasifikácie (FN + FP) / súčet všetkých

test accuracy je úspešnosť klasifikácie (TP + TN) / súčet všetkých

In [29]:
f = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 42742 samples, validate on 4750 samples
Epoch 1/20
42742/42742 [==============================] - 1570s 37ms/step - loss: 0.3263 - accuracy: 0.8612 - val_loss: 0.1814 - val_accuracy: 0.9339

Epoch 00001: val_accuracy improved from -inf to 0.93389, saving model to model_9_1.hdf5
Epoch 2/20
42742/42742 [==============================] - 1532s 36ms/step - loss: 0.1892 - accuracy: 0.9316 - val_loss: 0.1503 - val_accuracy: 0.9482

Epoch 00002: val_accuracy improved from 0.93389 to 0.94821, saving model to model_9_1.hdf5
Epoch 3/20
42742/42742 [==============================] - 1512s 35ms/step - loss: 0.1318 - accuracy: 0.9562 - val_loss: 0.1076 - val_accuracy: 0.9674

Epoch 00003: val_accuracy improved from 0.94821 to 0.96737, saving model to model_9_1.hdf5
Epoch 4/20
42742/42742 [==============================] - 1511s 35ms/step - loss: 0.0998 - accuracy: 0.9676 - val_loss: 0.1232 - val_accuracy: 0.9667

Epoch 00004: val_accuracy did not improve from 0.96737
Epoch 5/20
42742/4274

Načítanie modelu

In [13]:
print("Loading model....")
model = load_model('model_9_1.hdf5')

Loading model....




Vyhodnotenie test loss a accuracy

In [31]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02359017183333063
Test accuracy: 0.9919283390045166


In [32]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.15845295498294046
Test accuracy: 0.9818947315216064


Predikcia

In [14]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [15]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [16]:
# TN / FN
# FP / TP
print(matrix)

[[2409    5]
 [  41 2295]]


Classification Report

In [37]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2398
           1       0.99      0.97      0.98      2352

   micro avg       0.98      0.98      0.98      4750
   macro avg       0.98      0.98      0.98      4750
weighted avg       0.98      0.98      0.98      4750
 samples avg       0.98      0.98      0.98      4750



In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 64)        102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 64)       